## Hash Join

In [13]:
import polars as pl
from collections import defaultdict
import sys
from pprint import pprint

def hash_join(table1: pl.DataFrame, index1, table2: pl.DataFrame, index2):
    new_headers = []
    for c in table1.columns:
        if c in table2.columns:
            new_headers.append(c + '_l')
        else:
            new_headers.append(c)
    for c in table2.columns:
        if c in table1.columns:
            new_headers.append(c + '_r')
        else:
            new_headers.append(c)

    table1, table2 = table1.rows(), table2.rows()
    h = defaultdict(list)
    # hash phase
    for s in table1:
        h[s[index1]].append(s)
    # join phase
    pprint(h)
    res = [(s + r) for r in table2 for s in h[r[index2]]]

    return pl.DataFrame._from_records(res, schema=new_headers)
    

df1 = pl.DataFrame({
    'age': [27, 18, 28, 18, 28],
    'name': ["Jonah", "Alan", "Glory", "Popeye", "Alan"]
    })

df2 = pl.DataFrame({
    'name': ["Jonah", "Jonah", "Alan", "Alan", "Glory"],
    'word': ['Whales', 'Spiders', 'Ghosts', 'Zombies', 'Buffy']
})

hash_join(df1, 1, df2, 0)

defaultdict(<class 'list'>,
            {'Alan': [(18, 'Alan'), (28, 'Alan')],
             'Glory': [(28, 'Glory')],
             'Jonah': [(27, 'Jonah')],
             'Popeye': [(18, 'Popeye')]})


age,name_l,name_r,word
i64,str,str,str
27,"""Jonah""","""Jonah""","""Whales"""
27,"""Jonah""","""Jonah""","""Spiders"""
18,"""Alan""","""Alan""","""Ghosts"""
28,"""Alan""","""Alan""","""Ghosts"""
18,"""Alan""","""Alan""","""Zombies"""
28,"""Alan""","""Alan""","""Zombies"""
28,"""Glory""","""Glory""","""Buffy"""


In [ ]:
def hash_join_with_partitions(table1: pyarrow.dataset, index1, table2: pyarrow.dataset, index2):
    '''implement hash join that accepts table partitions
    
    the hash phase should wrap a for loop above `for s in table1` for all the partitions and store the join values in the hash
    '''

    hash_table = defaultdict(list)
    result = []
    # hash phase
    for batch in table1.to_batches():
        rows = pl.DataFrame._from_arrow(batch).rows()
        for row in rows:
            hash_table[row[index1]].append(row)

    # join phase
    for batch in table2.to_batches():
        rows = pl.DataFrame._from_arrow(batch).rows()
        for row in rows:
            for entry in hash_table[row[index2]]:
                result.append(entry + row)

## External Merge Sort

In [170]:
import names
import random

random.seed(123)
data_1 = [(names.get_first_name(), random.randint(0, 100)) for i in range(10)]
data_2 = [(names.get_first_name(), random.randint(0, 100)) for i in range(10)]
data_3 = [(names.get_first_name(), random.randint(0, 100)) for i in range(10)]
data_4 = [(names.get_first_name(), random.randint(0, 100)) for i in range(10)]

data_files = [data_1, data_2, data_3, data_4]

# sort phase
for data in data_files:
    data.sort(key=lambda x: x[1])

# merge phase
def current_tuple(idx, data: list) -> list:
    if idx < len(data):
        return [data[idx]]
    else:
        return []

i = j = k = l = 0
out_buffer = []
while i < len(data_1) or j < len(data_2) or k < len(data_3) or l < len(data_4):
    current_min = min(current_tuple(i, data_1) + current_tuple(j, data_2) + \
        current_tuple(k, data_3) + current_tuple(l, data_4), key=lambda x: x[1])
    if i < len(data_1) and current_min == data_1[i]:
        out_buffer.append(data_1[i])
        i += 1
    elif j < len(data_2) and current_min == data_2[j]:
        out_buffer.append(data_2[j])
        j += 1
    elif k < len(data_3) and current_min == data_3[k]:
        out_buffer.append(data_3[k])
        k += 1
    else:
        out_buffer.append(data_4[l])
        l += 1

out_buffer == sorted(data_1 + data_2 + data_3 + data_4, key=lambda x: x[1])

True

implement with polars dataframes

In [2]:
import polars as pl
import names
import random
import os

## setup
random.seed(999)

df1 = pl.DataFrame({
    'name': [names.get_first_name() for i in range(10)],
    'age': [random.randint(0, 100) for i in range(10)]
})
df2 = pl.DataFrame({
    'name': [names.get_first_name() for i in range(10)],
    'age': [random.randint(0, 100) for i in range(10)]
})
df3 = pl.DataFrame({
    'name': [names.get_first_name() for i in range(10)],
    'age': [random.randint(0, 100) for i in range(10)]
})
df4 = pl.DataFrame({
    'name': [names.get_first_name() for i in range(10)],
    'age': [random.randint(0, 100) for i in range(10)]
})

if not os.path.exists('./data_1.parquet'):
    df1.write_parquet('./data_1.parquet')
if not os.path.exists('./data_2.parquet'):
    df2.write_parquet('./data_2.parquet')
if not os.path.exists('./data_3.parquet'):
    df3.write_parquet('./data_3.parquet')
if not os.path.exists('./data_4.parquet'):
    df4.write_parquet('./data_4.parquet')

#############--------------###################

data_files = sorted([f for f in os.listdir('.') if f.endswith('parquet')])
schema = list(pl.read_parquet_schema(data_files[0]).keys()) # get schema/column names

'''
# sort
for file in data_files:
    data = pl.read_parquet('./' + file).rows()
    data.sort(key=lambda x: x[1])
    data = pl.DataFrame(data, schema=schema)
    if not os.path.exists('./' + file.split('.')[0] + '_sorted.parquet'):
        data.write_parquet('./' + file.split('.')[0] + '_sorted.parquet')
'''

# merge
n_buffers = 4
len_file = 10
batch_size = len_file / (n_buffers + 1)

In [59]:
import pyarrow as pa
import pyarrow.parquet as pq

pf1 = pq.ParquetFile('./data_1_sorted.parquet')
iterator_1 = pf1.iter_batches(batch_size=1)

pf2 = pq.ParquetFile('./data_2_sorted.parquet')
iterator_2 = pf2.iter_batches(batch_size=1)

merged_data = []


data_1, data_2 = next(iterator_1, False), next(iterator_2, False)

while data_1 and data_2:
    current_min = min(data_1.column('age')[0].as_py(), data_2.column('age')[0].as_py())
    if current_min == (data_1.column('age')[0].as_py()):
        merged_data.append(pl.from_arrow(data_1).rows()[0])
        data_1 = next(iterator_1, False)
    else:
        merged_data.append(pl.from_arrow(data_1).rows()[0])
        data_2 = next(iterator_2, False)

In [60]:
merged_data

[('Jamie', 11),
 ('Jamie', 11),
 ('Jamie', 11),
 ('Christina', 21),
 ('Christina', 21),
 ('Clarence', 23),
 ('Clarence', 23),
 ('Elsie', 26),
 ('Chad', 42),
 ('Chad', 42),
 ('Chad', 42),
 ('Chad', 42),
 ('Gladys', 64),
 ('Jennifer', 70),
 ('Jennifer', 70),
 ('Rocco', 76),
 ('Rocco', 76),
 ('Jean', 82)]

In [61]:
pl.DataFrame(merged_data)

column_0,column_1
str,i64
"""Jamie""",11
"""Jamie""",11
"""Jamie""",11
"""Christina""",21
"""Christina""",21
"""Clarence""",23
"""Clarence""",23
"""Elsie""",26
"""Chad""",42
